In [1]:
import pandas as pd
import bioframe as bf
import re


### Load Gencode mm39 annotation as a bed file

In [5]:
mm39_df = pd.read_csv('./gencode.vM32.annotation.gff3.bed', delimiter='\t',names=['chrom','start','end','strand','gene_type'])
mm39_df.head()

,chrom,start,end,strand,gene_type
0,chr1,3143475,3144545,+,TEC
1,chr1,3172238,3172348,+,snRNA
2,chr1,3276123,3741721,-,protein_coding
3,chr1,3322979,3323459,+,processed_pseudogene
4,chr1,3435953,3438772,-,TEC


### Merge overlapping gene intervals

In [6]:
mm39_merged_df = bf.merge(mm39_df, min_dist=0)
mm39_merged_df['int_len'] = mm39_merged_df['end'] - mm39_merged_df['start'] + 1
mm39_merged_df

,chrom,start,end,n_intervals,int_len
0,chr1,3143475,3144545,1,1071
1,chr1,3172238,3172348,1,111
2,chr1,3276123,3741721,10,465599
3,chr1,3750377,3752011,1,1635
4,chr1,3822232,3824583,1,2352
...,...,...,...,...,...
38999,chrY,90614769,90617133,1,2365
39000,chrY,90676614,90678894,1,2281
39001,chrY,90763695,90774754,2,11060
39002,chrY,90796006,90827734,1,31729


### Load regions of interest (ri) sent by Leah

In [7]:
ri_df = pd.read_csv('./regions_of_interest.bed', delimiter='\t', names=['chrom','start','end'])
ri_df['start'] = ri_df['start'] - 1
ri_df

,chrom,start,end
0,chr7,3180405,6180408
1,chr7,72537504,75537507
2,chr7,141894604,144894607
3,chr19,3309855,6309858
4,chr19,58300303,61300306
5,chr1,3050015,8050020
6,chr1,97547943,100547946
7,chr1,190045869,195045874


### Calculate overlapping coverage of annotation vs regions of interest

In [8]:
ri_coverage_df = bf.coverage(ri_df, mm39_merged_df)
display(ri_coverage_df)


,chrom,start,end,coverage
0,chr7,3180405,6180408,1459324
1,chr7,72537504,75537507,1434513
2,chr7,141894604,144894607,2019081
3,chr19,3309855,6309858,2206148
4,chr19,58300303,61300306,1365835
5,chr1,3050015,8050020,2079578
6,chr1,97547943,100547946,1293648
7,chr1,190045869,195045874,2498221


### Estimate RI-length (int_len) and corresponding percent coverage (cov_per)

In [9]:
ri_coverage_df['int_len'] = ri_coverage_df['end'] - ri_coverage_df['start'] + 1
ri_coverage_df['cov_per'] = round((ri_coverage_df['coverage'] / ri_coverage_df['int_len']) * 100, ndigits=1)
ri_coverage_df

,chrom,start,end,coverage,int_len,cov_per
0,chr7,3180405,6180408,1459324,3000004,48.6
1,chr7,72537504,75537507,1434513,3000004,47.8
2,chr7,141894604,144894607,2019081,3000004,67.3
3,chr19,3309855,6309858,2206148,3000004,73.5
4,chr19,58300303,61300306,1365835,3000004,45.5
5,chr1,3050015,8050020,2079578,5000006,41.6
6,chr1,97547943,100547946,1293648,3000004,43.1
7,chr1,190045869,195045874,2498221,5000006,50.0


**Column descriptions:**
* coverage: coverage in bp of transcriptionally active regions on interval of interest
* int_len: length in bp of the interval of interest
* cov_per: percent coverage of transcriptionally active regions relative to interval of interest

### Save final table to csv file

In [10]:
ri_coverage_df.to_csv('coding_coverage.csv', index=False)

### Remove pseudogenes from annotation

In [19]:
mm39_nopseud_df = mm39_df[mm39_df['gene_type'].str.find('pseud') == -1].copy()

### Calculate RI-length (int_len) and corresponding percent coverage (cov_per) as before but without pseudogenes

In [20]:
# Merge coordinates of overlapping genes
mm39_nopseud_merged_df = bf.merge(mm39_nopseud_df, min_dist=0)
mm39_nopseud_merged_df['int_len'] = mm39_nopseud_merged_df['end'] - mm39_nopseud_merged_df['start'] + 1

# Calculate overlapping coverage of annotation vs regions of interest
ri_coverage_nopseud_df = bf.coverage(ri_df, mm39_nopseud_merged_df)

# Estimate RI-length (int_len) and corresponding percent coverage (cov_per)
ri_coverage_nopseud_df['int_len'] = ri_coverage_nopseud_df['end'] - ri_coverage_nopseud_df['start'] + 1
ri_coverage_nopseud_df['cov_per'] = round((ri_coverage_nopseud_df['coverage'] / ri_coverage_nopseud_df['int_len']) * 100, ndigits=1)
ri_coverage_nopseud_df


,chrom,start,end,coverage,int_len,cov_per
0,chr7,3180405,6180408,1409548,3000004,47.0
1,chr7,72537504,75537507,1429235,3000004,47.6
2,chr7,141894604,144894607,2018137,3000004,67.3
3,chr19,3309855,6309858,2159948,3000004,72.0
4,chr19,58300303,61300306,1335021,3000004,44.5
5,chr1,3050015,8050020,2037351,5000006,40.7
6,chr1,97547943,100547946,1289063,3000004,43.0
7,chr1,190045869,195045874,2493248,5000006,49.9


### Save final table to csv file

In [21]:
ri_coverage_nopseud_df.to_csv('coding_coverage_no_pseudogenes.csv', index=False)